### BOSTON CRIMES REPORT 

- Create database crimes_db
- With table boston_crimes
- Appropriate datatypes from boston.csv
- Create table inside schema crimes
- readonly: data analyst , readwrite: data scientist




In [162]:
import psycopg2
import csv

In [163]:
def connection(dbname,user,password=None):
    conn = psycopg2.connect(dbname=dbname,user=user,password=None)
    conn.autocommit = True
    cur = conn.cursor()
    return conn, cur

In [164]:
conn, cur = connection("dq","dq")

In [ ]:
cur.execute("CREATE DATABASE crime_db;")

In [166]:
conn.commit()
conn.close()

In [167]:
conn, cur = connection("crime_db", "dq")

### Create Schema

In [22]:
cur.execute("CREATE SCHEMA crimes;")

### Choice the best data type

In [34]:
with open("boston.csv") as f:
    f = csv.reader(f)
    col_headers = next(f)
    first_row = next(f)

In [58]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [63]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename) as f:
        f = csv.reader(f)
        col_headers = next(f)
        elements = set(row[col_index] for row in f)
        return elements
    

In [81]:
for i in range(6):
    print(col_headers[i], len(get_col_value_set("boston.csv", i)))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177


In [69]:
index_description = col_headers.index("description")

In [74]:
len(max(get_col_value_set("boston.csv", index_description)))

32

In [77]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [78]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### Create TYPE & tables

In [86]:
cur.execute("CREATE TYPE day_date AS ENUM ('Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday');")

In [106]:
cur.execute("""CREATE TABLE crimes.bostons_crimes (
           incident_number INT PRIMARY KEY,
           offense_code INT,
           description TEXT,
           date date,
           day_of_week day_date,
           lat float8,
           long float8);""")
           

In [107]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.bostons_crimes FROM STDIN CSV HEADER;", f)

In [109]:
cur.execute("SELECT * FROM crimes.bostons_crimes;")
cur.fetchone()

(1,
 619,
 'LARCENY ALL OTHERS',
 datetime.date(2018, 9, 2),
 'Sunday',
 42.35779134,
 -71.13937053)

### Privileges

In [114]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [115]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### Creating User Groups

In [126]:
cur.execute("CREATE GROUP readonly WITH NOLOGIN;")

In [127]:
cur.execute("CREATE GROUP readwrite WITH NOLOGIN;")

In [132]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO GROUP readonly;")

In [129]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO GROUP readwrite;")

In [134]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO GROUP readonly;")

In [135]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO GROUP readwrite;")

In [136]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO GROUP readonly;")

In [138]:
cur.execute("GRANT SELECT,INSERT,DELETE,UPDATE ON ALL TABLES IN SCHEMA crimes TO GROUP readwrite;")

### Creating Users

In [139]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

In [140]:
cur.execute("GRANT data_analyst to readonly;")

In [141]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

In [145]:
cur.execute("GRANT data_scientist TO readwrite;")

In [153]:
cur.execute("""
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee = 'readwrite'
;
""")

In [ ]:
cur.fetchall()